### Spark-29351 Reproduction
---  
#### Load Session Data

In [ ]:
import scripts.session as session

session_data = session.load()

#### Copy the code repo to node


In [ ]:
from chi import ssh
import scripts.ssh_helper as ssh_helper

In [ ]:
# put the trovi to node

with ssh.Remote(session_data["floating_ip"]) as conn:
    conn.run('sudo mkdir -p /mnt/spark-exp; sudo chown $USER -R /mnt; cd /mnt/spark-exp; rm -rf /mnt/spark-exp/spark_bug_reproduce; mkdir -p /mnt/spark-exp; cd /mnt/spark-exp; git clone https://github.com/QinPR/spark_bug_reproduce.git; cd spark_bug_reproduce; git checkout Bug29351')

### 1. Buggy Version
#### 1.1. Set up the Spark evironment

In [ ]:
with ssh.Remote(session_data["floating_ip"]) as conn:
    conn.run('cd /mnt/spark-exp/spark_bug_reproduce; bash set_spark_env.sh')

#### 1.2. Run the Reproduction Experiments

In [ ]:
with ssh.Remote(session_data["floating_ip"]) as conn:
    conn.run('cd /mnt/spark-exp/spark_bug_reproduce; bash reproduce_bug29351.sh')

#### 1.3. Display Reproduction Graph [Buggy version]

In [ ]:
import os

def get_dir(conn, source, target):
    ''' Downloads the contents of the source directory from the remote to the target path locally.
        All subdirectories in source are created under target.
    '''
    if not os.path.exists(target):
        os.makedirs(target)
    
    result = conn.run(f'ls -A {source}', hide=True)
    for item in result.stdout.splitlines():
        remote_path = f"{source}/{item}"
        local_path = os.path.join(target, item)
        
        if conn.run(f'test -d {remote_path}', warn=True).ok:
            if not os.path.exists(local_path):
                os.makedirs(local_path)
            get_dir(conn, remote_path, local_path)
        else:
            conn.get(remote_path, local_path)

In [ ]:
from IPython.display import Image

# Gather the resulting graph
with ssh.Remote(session_data["floating_ip"]) as conn:
    conn.run('cd /mnt/spark-exp/spark_bug_reproduce; mkdir bug_plot; mv line_chart.png bug_plot/')
    get_dir(conn, "/mnt/spark-exp/spark_bug_reproduce/bug_plot", ".")

Image(filename="./line_chart.png")

### 2. Patched Version
#### 2.1. Compile the Spark

In [ ]:
with ssh.Remote(session_data["floating_ip"]) as conn:
    conn.run('sudo mkdir -p /mnt/spark-exp; sudo chown $USER -R /mnt; cd /mnt/spark-exp; rm -rf /mnt/spark-exp/spark_bug_reproduce; mkdir -p /mnt/spark-exp; cd /mnt/spark-exp; git clone https://github.com/QinPR/spark_bug_reproduce.git; cd spark_bug_reproduce; git checkout Bug29351_fix')


with ssh.Remote(session_data["floating_ip"]) as conn:
    conn.run('cd /mnt/spark-exp/spark_bug_reproduce; git checkout Bug29351_fix; bash set_spark_env.sh')

#### 2.2. Run the Reproduction Experiments

In [ ]:
with ssh.Remote(session_data["floating_ip"]) as conn:
    conn.run('cd /mnt/spark-exp/spark_bug_reproduce; bash reproduce_bug29351.sh')

#### 1.3. Display Reproduction Graph [Patched version]

In [ ]:
from IPython.display import Image

# Gather the resulting graph
with ssh.Remote(session_data["floating_ip"]) as conn:
    conn.run('cd /mnt/spark-exp/spark_bug_reproduce; mkdir bug_plot; mv line_chart_fix.png bug_plot/')
    get_dir(conn, "/mnt/spark-exp/spark_bug_reproduce/bug_plot", ".")

Image(filename="./line_chart_fix.png")